# 📱 AIDL 계산기 실습 프로젝트 - Android Studio 기반
Android Studio Narwhal 2025.1.1에서 AIDL(Android Interface Definition Language)을 이용한 간단한 계산기 서비스 실습 예제입니다.

## 📁 프로젝트 구조

AidlCalculator/  
├── app/  
│   ├── src/  
│   │   ├── main/  
│   │   │   ├── aidl/  
│   │   │   │   └── com/example/aidldemo/ICalcService.aidl  
│   │   │   ├── java/com/example/aidldemo/  
│   │   │   │   ├── MainActivity.java  
│   │   │   │   └── CalcService.java  
│   │   │   └── AndroidManifest.xml  
│   ├── build.gradle.kts  
├── build.gradle.kts  
└── settings.gradle.kts  


## 1️⃣ Android Studio에서 새 프로젝트 생성
1. **New Project** > **No Activity** 선택
2. 프로젝트 이름: `CalcServiceApp`
3. 언어: `Java`
4. Minimum SDK: `API 28` 이상
5. Finish 클릭

## 2️⃣ AIDL 인터페이스 생성


### build.gradle.kts 파일 수정

In [ ]:
plugins {
    alias(libs.plugins.android.application)
    alias(libs.plugins.kotlin.android)
}

android {
    namespace = "com.example.aidldemo"
    compileSdk = 36

    defaultConfig {
        applicationId = "com.example.aidldemo"
        minSdk = 34
        targetSdk = 36
        versionCode = 1
        versionName = "1.0"

        testInstrumentationRunner = "androidx.test.runner.AndroidJUnitRunner"
    }

    buildTypes {
        release {
            isMinifyEnabled = false
            proguardFiles(
                getDefaultProguardFile("proguard-android-optimize.txt"),
                "proguard-rules.pro"
            )
        }
    }
    compileOptions {
        sourceCompatibility = JavaVersion.VERSION_11
        targetCompatibility = JavaVersion.VERSION_11
    }
    kotlinOptions {
        jvmTarget = "11"
    }

    sourceSets["main"].aidl.srcDirs("src/main/aidl")

    // ✅ 여기에 추가!
    buildFeatures {
        aidl = true
    }
}

dependencies {

    implementation(libs.androidx.core.ktx)
    implementation(libs.androidx.appcompat)
    implementation(libs.material)
    testImplementation(libs.junit)
    androidTestImplementation(libs.androidx.junit)
    androidTestImplementation(libs.androidx.espresso.core)
}

### 📄 AIDL 파일 추가
com/example/aidldemo 에서 마우스 우클릭 → New > AIDL > AIDL File

파일 이름: ICalcService

자동 생성된 ICalcService.aidl 열고 아래와 같이 수정:

In [ ]:
package com.example.aidldemo;

interface ICalcService {
    int add(int a, int b);
    int subtract(int a, int b);
    int multiply(int a, int b);
    int divide(int a, int b);
}

저장 시 자동으로 ICalcService.java 생성됨 (build/generated/...에 있음)

## 3️⃣ AIDL 서비스 구현 (`CalcService.java`)

### 📄 CalcService.java 생성

1. java/com.example.aidldemo 폴더에서 마우스 우클릭 → New > Java Class → 이름: CalcService

2. 아래 코드 입력:

In [ ]:
package com.example.aidldemo;

import android.app.Service;
import android.content.Intent;
import android.os.IBinder;
import android.os.RemoteException;
import android.util.Log;

public class CalcService extends Service {

    private static final String TAG = "CalcService";

    // AIDL 인터페이스 Stub 구현
    private final ICalcService.Stub mBinder = new ICalcService.Stub() {
        @Override
        public int add(int a, int b) throws RemoteException {
            Log.d(TAG, "add(" + a + ", " + b + ")");
            return a + b;
        }

        @Override
        public int subtract(int a, int b) throws RemoteException {
            Log.d(TAG, "subtract(" + a + ", " + b + ")");
            return a - b;
        }

        @Override
        public int multiply(int a, int b) throws RemoteException {
            Log.d(TAG, "multiply(" + a + ", " + b + ")");
            return a * b;
        }

        @Override
        public int divide(int a, int b) throws RemoteException {
            Log.d(TAG, "divide(" + a + ", " + b + ")");
            if (b == 0) {
                throw new ArithmeticException("Divide by zero");
            }
            return a / b;
        }
    };

    @Override
    public IBinder onBind(Intent intent) {
        Log.d(TAG, "Service bound");
        return mBinder;
    }

    @Override
    public void onDestroy() {
        Log.d(TAG, "Service destroyed");
        super.onDestroy();
    }
}


## 4️⃣ AndroidManifest.xml 설정

### 📄 AndroidManifest.xml에 서비스 등록

``` xml
<?xml version="1.0" encoding="utf-8"?>
<manifest xmlns:android="http://schemas.android.com/apk/res/android"
    package="com.example.aidldemo">

    <permission
        android:name="com.example.aidldemo.permission.BIND_CALC_SERVICE"
        android:protectionLevel="normal" />

    <application
        android:label="AIDL Java App">
        <activity android:name=".MainActivity"
            android:exported="true">
            <intent-filter>
                <action android:name="android.intent.action.MAIN"/>
                <category android:name="android.intent.category.LAUNCHER"/>
            </intent-filter>
        </activity>

        <!-- 🧠 서비스 등록 -->
        <service
            android:name=".CalcService"
            android:exported="true"
            android:permission="com.example.aidldemo.permission.BIND_CALC_SERVICE">

            <intent-filter>
                <action android:name="com.example.aidldemo.CalcService" />
            </intent-filter>
        </service>
    </application>
</manifest>
```


## 5️⃣ 클라이언트 앱 구현 (`MainActivity.java`)

### 📄 MainActivity.java 수정

In [ ]:
package com.example.aidldemo;

import android.app.Activity;
import android.content.ComponentName;
import android.content.Context;
import android.content.Intent;
import android.content.ServiceConnection;
import android.os.Bundle;
import android.os.IBinder;
import android.util.Log;

public class MainActivity extends Activity {
    ICalcService calcService;

    private ServiceConnection connection = new ServiceConnection() {
        public void onServiceConnected(ComponentName name, IBinder service) {
            calcService = ICalcService.Stub.asInterface(service);
            try {
                int result = calcService.subtract(10, 5);
                Log.d("CalcClient", "Result from service: " + result);
            } catch (Exception e) {
                Log.e("CalcClient", "Error: ", e);
            }
        }
        public void onServiceDisconnected(ComponentName name) {
            calcService = null;
        }
    };

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        Intent intent = new Intent("com.example.aidldemo.CalcService");
        intent.setPackage("com.example.aidldemo");
        bindService(intent, connection, Context.BIND_AUTO_CREATE);
    }
}


## 6️⃣ 실행 방법
- Android Studio에서 ctrl + F9 로 실행
- `Logcat`에서 결과 확인

**예상 로그:**
```bash
CalcClient: Result from service: 15
```